In [2]:
from rich.console import Console
from rich.table import Table
from tqdm import tqdm

console = Console()

# Create the table
table = Table(show_header=True, header_style="bold magenta")
table.add_column("Epoch", justify="center")
table.add_column("Training Loss", justify="center")
table.add_column("Validation Loss", justify="center")
table.add_column("RougeL", justify="center")
table.add_column("Bleu4", justify="center")
table.add_column("Bertscore F1", justify="center")

# Let's say after every epoch you append results:
for epoch in range(1, 11):
    # Dummy values
    training_loss = 0.5 / epoch
    validation_loss = 0.4 / epoch
    rougeL = 0.3 + 0.01 * epoch
    bleu4 = 0.09 + 0.005 * epoch
    bert_f1 = 0.86 + 0.002 * epoch

    table.add_row(
        str(epoch),
        f"{training_loss:.6f}",
        f"{validation_loss:.6f}",
        f"{rougeL:.6f}",
        f"{bleu4:.6f}",
        f"{bert_f1:.6f}",
    )
    print("hello")

# Use tqdm.write so progress bar is not broken
tqdm.write("")
console.print(table)


hello
hello
hello
hello
hello
hello
hello
hello
hello
hello



┏━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Epoch ┃ Training Loss ┃ Validation Loss ┃  RougeL  ┃  Bleu4   ┃ Bertscore F1 ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   1   │   0.500000    │    0.400000     │ 0.310000 │ 0.095000 │   0.862000   │
│   2   │   0.250000    │    0.200000     │ 0.320000 │ 0.100000 │   0.864000   │
│   3   │   0.166667    │    0.133333     │ 0.330000 │ 0.105000 │   0.866000   │
│   4   │   0.125000    │    0.100000     │ 0.340000 │ 0.110000 │   0.868000   │
│   5   │   0.100000    │    0.080000     │ 0.350000 │ 0.115000 │   0.870000   │
│   6   │   0.083333    │    0.066667     │ 0.360000 │ 0.120000 │   0.872000   │
│   7   │   0.071429    │    0.057143     │ 0.370000 │ 0.125000 │   0.874000   │
│   8   │   0.062500    │    0.050000     │ 0.380000 │ 0.130000 │   0.876000   │
│   9   │   0.055556    │    0.044444     │ 0.390000 │ 0.135000 │   0.878000   │
│  10   │   0.050000    │    0.040000     │ 0.400000 │ 0.140000 │   0.880000   │
└───────┴───────────────┴─────────────────┴──────────┴──────────┴──────────────┘

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
from tqdm.auto import tqdm
import time
from transformers.utils.notebook import NotebookTrainingTracker,NotebookProgressBar

tracker=NotebookTrainingTracker(10)
# tracker.add_child(NotebookProgressBar(10))
for i in range(10):
    tracker.update(i+1,comment=f"Epoch {i+1}/{10}",)
    tracker.write_line({"epoch": i, "loss": 0.1*i})
    tracker.display()
    pred_bar=tracker.add_child(10)
    # pred_bar=NotebookProgressBar(10)
    print("total:",pred_bar.total)
    for j in range(10):
        time.sleep(0.5)
        pred_bar.update(j+1)
        print("hello")
    pred_bar.close()
    tracker.remove_child()

    print("this")
    # time.sleep(0.5)
    # tqdm.write("Halfway done!")  # 😎 progress bar stays perfect
    # print("hello")

epoch,loss
0,0.000000
1,0.100000


total: 10
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
this
total: 10
hello
hello
hello
hello


KeyboardInterrupt: 

In [1]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

# 1. Load a tiny sample dataset
dataset = load_dataset("imdb", split="train[:2%]").train_test_split(test_size=0.2)

# 2. Load a tokenizer and model (BERT)
model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 3. Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.remove_columns(["text"])  # remove original text
encoded_dataset.set_format("torch")  # convert to PyTorch tensors

# 4. Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # evaluate after each epoch
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

# 5. Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
)

# 6. Train
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

d:\nlp\torchenv\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


disable:  False


Epoch,Training Loss,Validation Loss
1,0.000500,0.000313
2,0.000200,0.000139
3,0.000100,0.000105


values {'Training Loss': 0.0005, 'Validation Loss': 0.0003130123659502715, 'Epoch': 1}
values {'Training Loss': 0.0002, 'Validation Loss': 0.00013904072693549097, 'Epoch': 2}
values {'Training Loss': 0.0001, 'Validation Loss': 0.00010478297917870805, 'Epoch': 3}


TrainOutput(global_step=150, training_loss=0.01680489012117808, metrics={'train_runtime': 38.7257, 'train_samples_per_second': 30.987, 'train_steps_per_second': 3.873, 'total_flos': 78933316608000.0, 'train_loss': 0.01680489012117808, 'epoch': 3.0})